In [1]:
!python3 --version
#!python --version
#!jupyter --version

import inspect

from algebra import *
from gauss_iterative import *
from gauss_recursive import *
from example import *

Python 3.10.6


# Gaussian Elimination in plain python

## Motivation

Gaussian algorithm is constantly performed in scientific computing of matrix.
It is the basis of investigating many properties of matrix, like rank, inverse, kernel and of course the linear equation system. 

#TODO: Show example of a matrix in general and row_echelon form, containing some zero rows.

### Goal
Translate the often human oriented algorithm instructions into proper tasks working on a computer.

We have to find representations in code for concepts like "cross the first column off mentally" or
"swap the row with a *suitable* row". 

We need to judge whether it makes sense to stick to the recipe (global vs. local vision) or if there are some (heuristic) shortcuts for the computer.

![](gauss1.jpeg)
_Source: Page 60 of the textbook of Fraleigh and Beauregard on Linear Algebra (3rd edition, 1995)._

## Implementation

### Datatypes for type hints
```python
from fractions import Fraction
F = Fraction | float | int 
R = list[F]
M = list[R]    # 0-based indexing
```

### Algebra on Matrix
```python
# Scalar Multiplication
def scalar_mult(M1: M, k: F) -> M
# Addition
def add(M1: M, M2: M) -> M
# Multiplication
def mult(M1: M, M2: M) -> M
# Take the cth-column
def column(M1: M, c: int) -> R
```

### Actions and their Elementary Matrices
```python
# Identity Matrix
def I(n: int) -> M
# Swap two rows
def S(n: int, r1: int, r2: int) -> M
# Multiply a row a times
def M(n: int, r1: int, a: F) -> M
# Add a times of r2 into r1
def A(n: int, r1: int, r2: int, a: F) -> M
```

In [15]:
simpleM = [ [1,2,3], [0,0,1] ]

show(simpleM)
show(inverse(simpleM))

1	2	3
0	0	1

1	-3
0	1



In [6]:
show(I(len(simpleM)))

1	0
0	1



In [17]:
show(mult(I(2), simpleM))

show(inverse(simpleM))
assert mult(I(2), simpleM) == mult(inverse(simpleM), I(len(inverse(simpleM))))

1	2	3
0	0	1

1	-3
0	1



AssertionError: 

In [11]:
show(mult(M(2, 0, 2), simpleM))

2	4	6
0	0	1



## Demos
Let's first show some examples.

In [2]:
show(Real_Matrix)

3.50	-2.00	1.00
-3.10	5.60	0.00
4.70	0.00	-1.30



In [3]:
Real_echelon, _, Elementary_trace = gauss_algorithm_iterative(Real_Matrix, True)
show(Real_echelon)

3.50	-2.00	1.00
0.00	3.83	0.89
0.00	0.00	-3.26



In [4]:
steps = StepByStep(Real_Matrix, Elementary_trace)

In [ ]:
try:
    next(steps)
except:
    print("This is the end of the algorithm!")

In [116]:
show(Rational_Matrix)

2	1	0	0
3	5	-1	-3
0	3	-1	0
1	4	0	-2



In [117]:
Rational_reduced_echelon, rank, Elementary_trace = normalize(Rational_Matrix, True)
print("Rank of this matrix is:", rank)
show(Rational_reduced_echelon)

Rank of this matrix is: 4
1	0	0	0
0	1	0	0
0	0	1	0
0	0	0	1



In [118]:
steps = StepByStep(Rational_Matrix, Elementary_trace)

In [132]:
try:
    next(steps)
except:
    print("This is the end of the algorithm!")

This is the end of the algorithm!


We can compute the inverse of an invertible matrix by simply multipling all elementary matrices together.

In [133]:
# But be careful that the matrix multiplications don't commute!
Elementary_trace.reverse()
Inverse = reduce(mult, Elementary_trace, I(rank))
show(Inverse)
show(mult(Rational_Matrix, Inverse))

8/19	2/19	-2/19	-3/19
3/19	-4/19	4/19	6/19
9/19	-12/19	-7/19	18/19
10/19	-7/19	7/19	1/19

1	0	0	0
0	1	0	0
0	0	1	0
0	0	0	1



## Details of the Implementation

### Functional Paradigm / Category Theory

- `map` applies a function on `a` to a list of `a`s:
```map(increment, [1,2,3]) -> [increment(2), increment(3), increment(4)] -> [2,3,4]```
- `reduce` / `fold` combines two elements of a list and accumulates the result
  `fold(add, [1,2,3]) -> add(add(1,2),3)` -> 6

### Iterative Version

Perform Gauss algorithm (iterative)

```python
def gauss_algorithm_iterative(m: M, is_traced=False) -> tuple[M, int, list[M]]
```

1. Get most-left column with non-zero values, find best row for first column, otherwise ignore this column by increasing now_column
2. If the top-row value is zero, then swap now_row with last non-zero row (or put to bottom using nullrow_cnt)
3. Make zeroes below the pivot (by adding the respective inverse multiple)
4. Perform 1-3 with remaining rows.

It returns the reduced matrix(in echelon form), the rank and the trace of operations

```python
    nullrow_cnt = 0
    now_row = 0
    now_column = 0
    row_dim = len(m)
    trace = []
    while now_row < row_dim - nullrow_cnt:
        (pivot_index, pivot) = get_pivot(m[now_row])

        if pivot_index == None:  # it's a nullrow
            swap = S(row_dim, now_row, row_dim - 1 - nullrow_cnt)
            trace.append(swap)
            m = mult(swap, m)
            nullrow_cnt += 1
        else:
            if pivot_index > now_column:  # there's still better pivot column at left
                better_candidate = find_better_candidate(
                    m, now_column, pivot_index, now_row
                )
                swap = S(row_dim, now_row, better_candidate)
                trace.append(swap)
                m = mult(swap, m)
                (pivot_index, pivot) = get_pivot(
                    m[now_row]
                )  # after swapping, we must get the pivot again
            col = column(m, pivot_index)
            scalar = list(map(lambda c: c / pivot, col))
            for k in range(now_row + 1, row_dim - nullrow_cnt):
                if (
                    -scalar[k] < 0 or -scalar[k] > 0
                ):  # if already 0, we don't need to do anything
                    addition = A(row_dim, k, now_row, -scalar[k])
                    trace.append(addition)
                    m = mult(addition, m)
            now_row += 1
            now_column += 1
    if is_traced:
        return (m, row_dim - nullrow_cnt, trace)
    else:
        return (m, row_dim - nullrow_cnt, [])
```

Let's check how `get_pivot` works:

In [76]:

R1 = [3, 4, 0]
print(get_pivot(R1), '\t is (index, value) of the pivot of row ', R1)

R2 = [0, 0, 1]
print(get_pivot(R2), '\t is (index, value) of the pivot or row ', R2)

(0, 3) 	 is (index, value) of the pivot of row  [3, 4, 0]
(2, 1) 	 is (index, value) of the pivot or row  [0, 0, 1]


And also `find_better_candidate`:

In [ ]:
M1 = [[0, 0, 5], [0, 1, 0], [2, 4, 0]]
show(M1)

In [ ]:
print(find_better_candidate(M1, 0, 2, 0))

Normalize makes the matrix into a reduced echelon matrix.

```python
def normalize(m: M, is_traced=False) -> tuple[M, int, list[M]]
```

It applies the gauss algorithm first, then it goes from bottom(non-zero row) to top with following steps:

1. Normalize the pivot element of this row into 1
2. Make the element above the pivot zero by adding the respective additive inverse.
3. Repeat 1-2 on the above row till the first row.

It returns a tuple with reduced matrix, the rank of the matrix and trace of operations

```python
    row_dim = len(m)
    (m, rank, trace) = gauss_algorithm_iterative(m, is_traced)
    pivots = get_pivots(m) #get all pivots to make it faster later
    for k in range(rank):
        mul = M(row_dim, rank - k - 1, 1 / pivots[rank - k - 1][1]) #normalizing
        if is_traced:
            trace.append(mul)
        m = mult(mul, m)
        col_index = pivots[rank - k - 1][0] #the column above this pivot should be cleared
        for r in range(rank - k - 1):
            if -m[r][col_index] < 0 or -m[r][col_index] > 0:
                addition = A(row_dim, r, rank - k - 1, -m[r][col_index]) #make the element zero
                if is_traced:
                    trace.append(addition)
                m = mult(addition, m)
    if is_traced:
        return (m, rank, trace)
    else:
        return (m, rank, [])
```

### Recursive Version

#TODO: Add code/comments

## Improvements / Refactoring

- The gauss function is quite big and deeply nested. It would be nice to have separate functions matching the steps in the algorithm.
- Implement different approaches, run performance tests and analyze. Maybe some heuristics can help to speed up unlucky cases (e.g. pivots all the way to the right)
- What about correctness?
  - e.g. numerical issues (convert int to fractions?)
- Make precise drawings of the data structures and operations
- Check properties and assert they hold within the algorithm.
- Represent `Matrix` as a class with properties and functions
- The bottleneck of this algorithm is the naive matrix multiplication (in a production environment it will be replaced by robust libraries and GPU acceleration)

In [ ]:
# yes
print(0.2 + 0.2 == 0.4)

# but
print(0.2 + 0.1 == 0.3)
print(0.2 - 0.2 == (((0.3 - 0.1) - 0.1) - 0.1))

In [ ]:
# See how long it takes!
gauss_algorithm_iterative(Big_Matrix)